# LQRにおけるサンプル効率

参考：
* [On the Sample Complexity of the Linear Quadratic Regulator](https://arxiv.org/abs/1710.01688)

モデルのダイナミクス行列$A$と$B$がわかっていればLQRでダイナミクスを解くことができますが，一般にはそうではありません．
わからないときは，ダイナミクスを推定したあとに最適制御を実行する必要があります．

表記：
$$
\begin{array}{ll}
\operatorname{minimize} & \mathbb{E}\left[\frac{1}{T} \sum_{t=1}^T x_t^* Q x_t+u_{t-1}^* R u_{t-1}\right] \\
\text { subject to } & x_{t+1}=A x_t+B u_t+w_t
\end{array}
$$
* $Q$は$n\times n$の行列
* $w_t\in \mathbb{R}^n$は平均０で分散$\Sigma_w$のガウシアンノイズ行列
* $G_T=\left[\begin{array}{llll}A^{T-1} B & A^{T-2} B & \ldots & B\end{array}\right] \quad$ and $\quad F_T=\left[\begin{array}{llll}A^{T-1} & A^{T-2} & \ldots & I_n\end{array}\right]$とする．


## ダイナミクスの推定

ガウシアンノイズによってシステムをロールアウトして，行列$A$と$B$を推定することを考えます．つまり，

$$
x_{t+1}^{(\ell)}=A x_t^{(\ell)}+B u_t^{(\ell)}+w_t^{(\ell)} \text { with } w_t^{(\ell)} \stackrel{\text { i.i.d. }}{\sim} \mathcal{N}\left(0, \sigma_w^2 I_n\right) \text { and } u_t^{(\ell)} \stackrel{\text { i.i.j. }}{\sim} \mathcal{N}\left(0, \sigma_u^2 I_p\right) \text {. }
$$

として，データセット$\left\{\left(x_t^{(\ell)}, u_t^{(\ell)}\right): 1 \leq \ell \leq N, 0 \leq t \leq T\right\}$を集め，システムを
$$
(\widehat{A}, \widehat{B}) \in \arg \min _{(A, B)} \sum_{\ell=1}^N \frac{1}{2}\left\|A x_{T-1}^{(\ell)}+B u_{T-1}^{(\ell)}-x_T^{(\ell)}\right\|_2^2
$$
として推定します．（これは解析を簡単にするために，ロールアウトで一個のサンプルが得られる状況だけ考えます．一方で，ロールアウトの全てのサンプルを使用する場合はBootstrap法として論文中に記載があるので，そちらを参照してください）

このとき，$N \geq 8(n+p)+16 \log (4 / \delta)$ならば確率$1-\delta$以上で
$$
\|\widehat{A}-A\|_2 \leq \frac{16 \sigma_w}{\sqrt{\lambda_{\min }\left(\sigma_u^2 G_T G_T^*+\sigma_w^2 F_T F_T^*\right)}} \sqrt{\frac{(n+2 p) \log (36 / \delta)}{N}}
$$
であり，また，
$$
\|\widehat{B}-B\|_2 \leq \frac{16 \sigma_w}{\sigma_u} \sqrt{\frac{(n+2 p) \log (36 / \delta)}{N}}
$$
が成り立つことを示しましょう．

---

**別の式表記**

証明の前に，式展開を簡単にするために表記を導入します．

$
\Theta:=\left[\begin{array}{ll}
A & B
\end{array}\right]^* \in \mathbb{R}^{(n+p) \times n}
$
および
$z_t:=\left[\begin{array}{l}x_t \\ u_t\end{array}\right] \in \mathbb{R}^{n+p}$
とします．

このとき，

$$
x_{t+1}^*=z_t^* \Theta+w_t^*
$$

が成立し，一度のロールアウトでは

$$
X:=\left[\begin{array}{c}
x_1^* \\
x_2^* \\
\vdots \\
x_T^*
\end{array}\right], Z:=\left[\begin{array}{c}
z_0^* \\
z_1^* \\
\vdots \\
z_{T-1}^*
\end{array}\right], W:=\left[\begin{array}{c}
w_0^* \\
w_1^* \\
\vdots \\
w_{T-1}^*
\end{array}\right]
$$

が得られます．

このときのシステムのダイナミクスは$X=Z\Theta + W$で与えられることに注意して，ロールアウトのデータセットを$X_N, Z_N, W_N$とすれば，（ここで$X_N=\left[\begin{array}{llll}x_T^{(1)} & x_T^{(2)} & \ldots & x_T^{(N)}\end{array}\right]^*$）

$$X_N=Z_N \Theta+W_N$$

で与えれれます．よって，$Z_N^*Z_N$がinvertibleであるとすれば，パラメータの推定は

$$
\widehat{\Theta}=\left(Z_N^* Z_N\right)^{-1} Z_N^* X_N=\Theta+\left(Z_N^* Z_N\right)^{-1} Z_N^* W_N
$$

であり，推定誤差は

$$
E:=\widehat{\Theta}-\Theta=\left(Z_N^* Z_N\right)^{-1} Z_N^* W_N
$$

になります．

---

$G_T$と$F_T$の定義を思い出しましょう．$G_T=\left[\begin{array}{llll}A^{T-1} B & A^{T-2} B & \ldots & B\end{array}\right], \quad F_T=\left[\begin{array}{llll}A^{T-1} & A^{T-2} & \ldots & I_n\end{array}\right]$だったので，

$$
x_T=G_T\left[\begin{array}{c}
u_0 \\
u_1 \\
\vdots \\
u_{T-1}
\end{array}\right]+F_T\left[\begin{array}{c}
w_0 \\
w_1 \\
\vdots \\
w_{T-1}
\end{array}\right]
$$

が成り立ちます．$u_t \sim \mathcal{N}\left(0, \sigma_u^2 I_p\right)$だったので，

$$
\left[\begin{array}{l}
x_T \\
u_T
\end{array}\right] \sim \mathcal{N}\left(0,\left[\begin{array}{cc}
\sigma_u^2 G_T G_T^*+\sigma_w^2 F_T F_T^* & 0 \\
0 & \sigma_u^2 I_p
\end{array}\right]\right)
$$

です．$F_T F_T^* \succ 0$なので，$\sigma_u$と$\sigma_w$が正であれば，これは退化していない分布になります．

これを踏まえて，次の補題を使います．

---

$\delta \in (0, 1)$かつ$N \geq 2 \log(1/\delta)$とする．
$f_k \in \mathbb{R}^m, g_k \in \mathbb{R}^n$を$f_k \sim \mathcal{N}\left(0, \Sigma_f\right)$ and $g_k \sim \mathcal{N}\left(0, \Sigma_g\right)$なる独立確率ベクトルとする．
このとき，確率$1-\delta$以上で，

$$
\left\|\sum_{k=1}^N f_k g_k^*\right\|_2 \leq 4\left\|\Sigma_f\right\|_2^{1 / 2}\left\|\Sigma_g\right\|_2^{1 / 2} \sqrt{N(m+n) \log (9 / \delta)}
$$

が成立する．

証明はBernsteinをいじるとでてくるよ．Appendix A参照．

---



この補題を使うと，$X$が$n_1\times N$の$\mathcal{N}(0, 1)$に従う確率分布， $Y$が$N\times n_2$の$\mathcal{N}(0, 1)$に従う確率分布とすると，確率$1-\delta$以上で，

$$
\|X Y\|_2 \leq 4 \sqrt{N\left(n_1+n_2\right) \log (9 / \delta)}
$$

が成り立つ．続いて，次の行列についての特異値のバウンドも便利です．

---

$X \in \mathbb{R}^{N \times n}$を$\mathcal{N}(0, 1)$に従う独立確率行列とすれば，確率$1-\delta$以上で，

$$
\sqrt{\lambda_{\min }\left(X^* X\right)} \geq \sqrt{N}-\sqrt{n}-\sqrt{2 \log (1 / \delta)}
$$

が成り立つ．

---

この２つを合体すると，次がわかります．

---

* $z_1, \dots, z_N \in \mathbb{R}^n$を$\mathcal{N}(0, \Sigma)$に従うとして，$\Sigma$をinvertibleとする．
* $Z^*:=\left[\begin{array}{lll}z_1 & \ldots & z_N\end{array}\right]$とする．
* $W\in \mathbb{R}^{N\times p}$の$\mathcal{N}(0, \sigma_w^2)$に従うとして，$Z$と独立とする．
* $E:=\left(Z^* Z\right)^{\dagger} Z^* W$とする．


このとき，$N \geq 8 n+16 \log (2 / \delta)$のとき，任意の行列$Q$に対して，確率$1-\delta$以上で

$$
\|Q E\|_2 \leq 16 \sigma_w\left\|Q \Sigma^{-1 / 2}\right\|_2 \sqrt{\frac{(n+p) \log (18 / \delta)}{N}}
$$

が成り立つ．

証明は論文参照．

---

以上を使えば，$\hat{A}-A$および$\hat{B}-B$の推定誤差のバウンドは簡単に出せます．



---

**バウンドのコメント**

$$
\|\widehat{A}-A\|_2 \leq \frac{16 \sigma_w}{\sqrt{\lambda_{\min }\left(\sigma_u^2 G_T G_T^*+\sigma_w^2 F_T F_T^*\right)}} \sqrt{\frac{(n+2 p) \log (36 / \delta)}{N}}
$$
$$
\|\widehat{B}-B\|_2 \leq \frac{16 \sigma_w}{\sigma_u} \sqrt{\frac{(n+2 p) \log (36 / \delta)}{N}}
$$

* 学習すべきパラメータは$n(n+p)$個ありますが，今回のバウンドは$(n+2p)$の依存なので，$n$と$p$についてtightです．
* $\lambda_{\min }\left(\sigma_u^2 G_T G_T^*+\sigma_w^2 F_T F_T^*\right)$は入力やノイズが状態全体にどれくらい増大されるか？の量になります．

## 推定したダイナミクスでの制御

ロバストな制御を実現するためにSystem Level Synthesisを使います．つまり，状態と入力をノイズだけで次のように表します．

$$
\begin{aligned}
& x_k=\sum_{t=1}^k(A+B K)^{k-t} w_{t-1}, \\
& u_k=\sum_{t=1}^k K(A+B K)^{k-t} w_{t-1}
\end{aligned}
$$

すると，$\Phi_x(k):=(A+B K)^{k-1}$および$\Phi_u(k):=K(A+B K)^{k-1}$とすれば，

$$\left[\begin{array}{l}x_k \\ u_k\end{array}\right]=\sum_{t=1}^k\left[\begin{array}{l}\Phi_x(k-t+1) \\ \Phi_u(k-t+1)\end{array}\right] w_{t-1}$$

が成り立ちます．$\left\{\Phi_x(k), \Phi_u(k)\right\}$は$K$による閉ループシステムのシステム応答と呼ばれます．
この$\Phi$について

$$
\Phi_x(k+1)=A \Phi_x(k)+B \Phi_u(k), \Phi_x(1)=I, \quad \forall k \geq 1
$$

が成り立ちます．

また，無限ホライゾンの場合はz変換を使って表記すると便利です．$\boldsymbol{\Phi}_x(z)=\sum_{k=1}^{\infty} \Phi_x(k) z^{-k}$とすると，上の制約は

$$
\left[\begin{array}{ll}
z I-A & -B
\end{array}\right]\left[\begin{array}{l}
\mathbf{\Phi}_x \\
\mathbf{\Phi}_u
\end{array}\right]=I
$$

として表現できます．このときのゲインは$\mathbf{u}=\mathbf{K} \mathbf{x}$ is given by $\mathbf{K}=\boldsymbol{\Phi}_u \boldsymbol{\Phi}_x^{-1}$です．
